In [47]:
import os
import json
import pandas
import numpy as np
import pickle
from sklearn import preprocessing
from sklearn.neural_network import MLPRegressor
import tensorflow as tf
import matplotlib.pyplot as plt
#import tensorflow.contrib.learn as skflow

% matplotlib inline

In [57]:
Ned = []
Med = []
NRd = []
MRd = []
fck = []
As = []
bh = []
a = []
xu = []
l0 = []
listdir = os.listdir("../../data_set/buckling")


for file in listdir:
    f = open("../../data_set/buckling/" + file)
    d = json.loads(f.read())
    f.close()

    for o in d:
        Ned.append(o["Ned"])
        #Med.append(o["Med"])
        #NRd.append(o["NRd"])
        MRd.append(o["MRd"])
        fck.append(o["fck"])
        As.append(o["As"])
        bh.append(o["bh"])
        #a.append(o["a"])
        #xu.append(o["xu"])
        l0.append(o["l0"])

rho = []
for i in range(len(As)):
    rho.append(As[i] / bh[i]**2)

In [59]:
df = pandas.DataFrame({
        "l0": l0,
        "Ned": Ned,
        "fck": fck,
        "bh": bh,
        "rho": rho,
        "MRd": MRd
    }
    )
print(df.shape)
df = df[df["fck"] == 20]
print(df.shape)
df.head(5)

(195333, 6)
(63168, 6)


,MRd,Ned,bh,fck,l0,rho
0,37582574,100000,250,20,2581.442640,0.004800
3,41767891,100000,275,20,3058.216260,0.003967
6,46007825,100000,300,20,3579.266366,0.003333
9,50355745,100000,325,20,4144.790181,0.002840
12,54748450,100000,350,20,4754.927524,0.002449


In [60]:
feat_df = df[[ "MRd", "Ned", "l0"]]
res_df = df[["bh"]]

scale_feat = preprocessing.StandardScaler()
scale_feat.fit(feat_df)
print(scale_feat.transform(feat_df))

scale_res = preprocessing.StandardScaler()
scale_res.fit(res_df)
df = df.sample(frac=1)


[[-2.01963088 -1.69879774 -0.47744239]
 [-1.99620105 -1.69879774 -0.30193669]
 [-1.97246546 -1.69879774 -0.11013232]
 ..., 
 [ 0.50585699 -0.59779675  0.87641894]
 [ 0.47269115 -0.59779675  1.01871937]
 [ 0.50272992 -0.59779675  1.17286487]]


In [61]:
split = int(45e3)
df_train = df[:split]
df_test = df[split:]
df_train.head(5)

print(df_train.shape, df_test.shape)
df_train.head(15)

(45000, 6) (18168, 6)


,MRd,Ned,bh,fck,l0,rho
155439,458087598,430000,625,20,7592.680035,0.003840
25899,500612672,1350000,725,20,5251.290825,0.000571
154534,272728849,430000,625,20,7148.675243,0.001664
139151,871491838,2860000,750,20,3856.423718,0.000533
35469,413596286,1480000,600,20,3463.268940,0.000833
93942,395978228,2250000,525,20,2167.420845,0.001088
8173,136114537,1130000,350,20,1414.505629,0.002449
13401,376705024,1180000,625,20,4200.044257,0.000768
38924,564132443,1500000,750,20,5325.030901,0.000533
151210,281939685,380000,625,20,7661.530969,0.001920


In [63]:
train_feat = scale_feat.transform(df_train[["MRd", "Ned", "l0"]])
train_label = df_train["bh"].as_matrix()
test_feat = scale_feat.transform(df_test[["MRd", "Ned", "l0"]])
test_label = df_test["bh"].as_matrix()

print("Training features:\n MRd, Ned, l0\n")
print(train_feat[:5])

print("\nTraining labels:\n bh\n")
print(train_label[:5])


print("\nTraining data:", train_feat.shape, train_label.shape)
print("\nTesting data:", test_feat.shape, test_label.shape)



Training features:
 MRd, Ned, l0

[[ 0.33439922 -1.29056142  1.36725013]
 [ 0.57245893 -0.15244803  0.50535858]
 [-0.703258   -1.29056142  1.203807  ]
 [ 2.64867841  1.71554242 -0.0081076 ]
 [ 0.08533231  0.00837234 -0.15283227]]

Training labels:
 bh

[625 725 625 750 600]

Training data: (45000, 3) (45000,)

Testing data: (18168, 3) (18168,)


In [109]:
X = train_feat
Y = train_label

# Tensorboard
def variable_summaries(var):
  """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
  with tf.name_scope('summaries'):
    mean = tf.reduce_mean(var)
    tf.summary.scalar('mean', mean)
    with tf.name_scope('stddev'):
      stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
    tf.summary.scalar('stddev', stddev)
    tf.summary.scalar('max', tf.reduce_max(var))
    tf.summary.scalar('min', tf.reduce_min(var))
    tf.summary.histogram('histogram', var)


In [65]:
batch = int(200)
tf.reset_default_graph()
graph = tf.Graph()
n_input = 3  # n variables
n_hidden_1 = 5
n_hidden_2 = 1
n_output = 1
beta = 0.001

learning_rate = 0.001

with graph.as_default():
    tf_train_feat = tf.placeholder(tf.float32, shape=(None, n_input))
    tf_train_label = tf.placeholder(tf.float32, shape=(None))
    

    tf_test_feat = tf.constant(test_feat, tf.float32)
    
    
    """
    Weights and biases. The weights matix' columns will be the output vector.
    
    * ndarray([rows, columns])
    * ndarray([in, out])
    
    tf.placeholder(None) and tf.placeholder([None, 3]) means that the row's size is not set. In the second 
    placeholder the columns are prefixed at 3.
    """
    W = {
        "layer_1": tf.Variable(tf.truncated_normal([n_input, n_hidden_1])),
        "layer_2": tf.Variable(tf.truncated_normal([n_hidden_1, n_hidden_2])),
        "layer_3": tf.Variable(tf.truncated_normal([n_hidden_2, n_output])),
    }
    
            
    b = {
        "layer_1": tf.Variable(tf.zeros([n_hidden_1])),
        "layer_2": tf.Variable(tf.zeros([n_hidden_2])),
        "layer_3": tf.Variable(tf.zeros([n_output])),
    }
    
    def computation(X):
        layer_1 = tf.nn.relu(tf.matmul(X, W["layer_1"]) + b["layer_1"])
        layer_2 = tf.nn.relu(tf.matmul(layer_1, W["layer_2"]) + b["layer_2"])
        layer_3 = tf.matmul(layer_2, W["layer_3"]) + b["layer_3"]
        return layer_2
    
    tf_prediction = computation(tf_train_feat)
    tf_test_prediction = computation(tf_test_feat)
    
    tf_loss = tf.reduce_mean(tf.square(tf_train_label - tf_prediction)) / 2#
    tf_loss = tf.reduce_mean(tf_loss + beta * (tf.nn.l2_loss(W["layer_1"]) + tf.nn.l2_loss(W["layer_2"])))
    tf_optimizer = tf.train.AdamOptimizer(learning_rate).minimize(tf_loss)
          

                             
#     # tensorboard
#     with tf.name_scope("layer_1"):
#         with tf.name_scope('weights'):
#             variable_summaries(W["layer_1"])
#             variable_summaries(b["layer_1"])
                             
#     with tf.name_scope("layer_2"):
#         with tf.name_scope('weights'):
#             variable_summaries(W["layer_2"])
#             variable_summaries(b["layer_2"])
    
#     with tf.name_scope("loss"):
#         tf.summary.scalar("loss", tf_loss)
        
    
#     writer = tf.summary.FileWriter("tensorboard", graph)
        
    init = tf.global_variables_initializer()

In [55]:
def accuracy(y_pred, y):
    a = 0
    for i in range(y.shape[0]):
        a += abs(1 - y_pred[i][0] / y[i])
    
    return round((1 - a / y.shape[0]) * 100, 3)

def accuracy_tensor(y_pred, y):
    a = 0
    for i in range(y.shape[0]):
        a += abs(1 - y_pred[i][0] / y[i])
    
    return round((1 - a / y.shape[0]) * 100, 3)

def shuffle_in_unison(a, b):
    assert len(a) == len(b)
    shuffled_a = np.empty(a.shape, dtype=a.dtype)
    shuffled_b = np.empty(b.shape, dtype=b.dtype)
    permutation = np.random.permutation(len(a))
    for old_index, new_index in enumerate(permutation):
        shuffled_a[new_index] = a[old_index]
        shuffled_b[new_index] = b[old_index]
    return shuffled_a, shuffled_b

In [110]:
# train_epoch = int(5e3)


# n_batch = int(X.shape[0] // batch)

# prev_acc = 0
# stable_count = 0

# session = tf.InteractiveSession(graph=graph)
# session.run(init)

# print("Initialized.\n No. of epochs: %d.\n No. of batches: %d." % (train_epoch, n_batch))

# # with tf.Session(graph=graph) as session:
# for epoch in range(train_epoch):
#     offset = (epoch * n_batch) % (Y.shape[0] - n_batch)
    
  
#     for i in range(n_batch):
#         x = X[offset:(offset + n_batch)]
#         y = Y[offset:(offset + n_batch)]
        
#         x, y = shuffle_in_unison(x, y)

#         feed_dict = {tf_train_feat: x, tf_train_label: y}
#         _, l, pred, pred_label = session.run([tf_optimizer, tf_loss, tf_prediction, tf_train_label], feed_dict=feed_dict)

#     if epoch % 8 == 0:
#         print("Epoch: %d. Batch' loss: %f" %(epoch, l))
#         test_pred = tf_test_prediction.eval(session=session)
        
#         acc_test = accuracy(test_pred, test_label)
#         acc_train = accuracy_tensor(pred, pred_label)
        
#         print("Accuracy train set %s%%" % acc_train)
#         print("Accuracy test set: %s%%" % acc_test)
#         print("Prediction:", pred[0], "Computed: ", test_label[0])



#         if stable_count > 15:
#             print("Finished")
#             break

m = MLPRegressor(verbose=True, hidden_layer_sizes=[15, 10]) 
m.fit(X, Y)

Iteration 1, loss = 157160.46078726
Iteration 2, loss = 131097.13019233
Iteration 3, loss = 70946.38806404
Iteration 4, loss = 23185.35825458
Iteration 5, loss = 10995.57021626
Iteration 6, loss = 8653.30551547
Iteration 7, loss = 7092.63949892
Iteration 8, loss = 5828.29661521
Iteration 9, loss = 4855.15453207
Iteration 10, loss = 4170.02427600
Iteration 11, loss = 3685.72026216
Iteration 12, loss = 3301.02634717
Iteration 13, loss = 2954.16472965
Iteration 14, loss = 2702.13131246
Iteration 15, loss = 2553.70220522
Iteration 16, loss = 2433.19623978
Iteration 17, loss = 2321.98395135
Iteration 18, loss = 2208.71695697
Iteration 19, loss = 2083.55552007
Iteration 20, loss = 1942.54041052
Iteration 21, loss = 1785.41652554
Iteration 22, loss = 1615.53293340
Iteration 23, loss = 1439.37696875
Iteration 24, loss = 1275.09141479
Iteration 25, loss = 1120.30734497
Iteration 26, loss = 984.54403244
Iteration 27, loss = 863.53088653
Iteration 28, loss = 758.26052124
Iteration 29, loss = 671.

C:\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=[15, 10], learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=True, warm_start=False)

In [55]:
session.close()

In [126]:
input = {
    "MRd": [200e6],
    "Ned": [1500e3],
    "l0": [2e3],
}

for i in input:
    input[i].append(0)

inp = pandas.DataFrame(input)
print(inp.head())

X = scale_feat.transform(inp[["MRd", "Ned", "l0"]])
a = m.predict(X)
print(a[0])
#a = session.run([tf_prediction], feed_dict={tf_train_feat: X})
print("\n\n", a[0].shape)
# h = a[0][0][0]
# ac = h**2
# as_ = ac * input["rho"][0]


# n = input["Ned"][0] / (ac * 20 + as_ * 435)
# ylim = 10.8 / n**0.5

# print(a[0][0])

# print("ylim: ", ylim)
#print("y: ", input["l0"][0] / ( h / 3.46 ))




         MRd      Ned    l0
0  200000000  1500000  2000
1          0        0     0
464.184749281


 ()
